##First we install Dependencies

In [5]:
!pip install -q transformers datasets


In [6]:
dataset_ckpt = 'zeroshot/twitter-financial-news-topic'
teacher_model_ckpt = 'odunola/bert-based_uncased-finetuned-financial-talk' #our already finetuned teacher model
student_model_ckpt = 'google/bert_uncased_L-4_H-512_A-8'

####Importing dependencies

In [7]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from transformers import AutoModelForSequenceClassification
from torch import nn
from torch import optim
from torch.nn import functional as F
from transformers import AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(teacher_model_ckpt)

In [9]:
test_list = ['W have a hope for the unknown'] * 5


In [10]:
student_tokens = tokenizer(test_list, max_length = 50, truncation = True, padding = True)
teacher_tokens = tokenizer(test_list, max_length = 50, truncation = True, padding = True)

In [11]:
student_tokens

{'input_ids': [[101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [12]:
teacher_tokens

{'input_ids': [[101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102], [101, 1059, 2031, 1037, 3246, 2005, 1996, 4242, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

###Preprocessing Data
We would be using the ag_news dataset comprised of 120k training samples and 7600 test samples. for our validation set we extract 12k samples from the original train set



In [13]:
data = load_dataset(dataset_ckpt)

In [14]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16990
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 4117
    })
})

In [44]:
train_test = data['train'].train_test_split(test_size = 0.1)
test_data = train_test['test']
train_data = train_test['train']
valid_data = data['validation']

In [45]:
def get_num_rows(dataset):
  return dataset.num_rows

print(f'Train set has {get_num_rows(train_data)} texts')
print(f'Valid set has {get_num_rows(valid_data)} texts')
print(f'Test set has {get_num_rows(test_data)} texts')

Train set has 15291 texts
Valid set has 4117 texts
Test set has 1699 texts


####Now we pull our tokenizer from the huggingface hub. since both are BERT Models we should be able to use the same tokenizzer for both student and teacher

In [46]:
tokenizer = AutoTokenizer.from_pretrained(teacher_model_ckpt)

In [47]:
#now we would utilise pytorch's Dataset andDataloader classes to create our dataset

class MyData(Dataset):
  def __init__(self, data):
    targets = data['label']
    texts = data['text']

    tokens = tokenizer(texts, return_tensors = 'pt', truncation = True, padding = 'max_length', max_length = 150)
    self.input_ids = tokens['input_ids']
    self.attention_mask = tokens['attention_mask']
    self.targets = torch.tensor(targets)
    self.length = len(texts)
  def __len__(self):
    return self.length
  def __getitem__(self, index):
    return self.input_ids[index], self.attention_mask[index], self.targets[index]

In [48]:
train_data = MyData(train_data)
valid_data = MyData(valid_data)
test_data = MyData(test_data)

####In Pytorch, dataloaders are iterators that make writing our training loops easier

In [49]:
# now we build our loaders
batch_size = 64
train_loader = DataLoader(train_data,batch_size = batch_size)
valid_loader = DataLoader(valid_data, batch_size = batch_size)
test_loader = DataLoader(test_data, batch_size = batch_size)

In [50]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


#### we define a function to help us compute accuracy as we train

In [51]:
from tqdm import tqdm
from time import perf_counter

In [52]:

# we define a function to help us compute accuracy as we train, we would also define another function to measure time ellapsed
def accuracy_score(batch, model):
  with torch.no_grad():
    outputs = model(
        batch[0].to(device),
        batch[1].to(device)
    )
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim = 1)
    class_predictions = torch.argmax(probabilities, dim = 1)
    acc = torch.mean((class_predictions == batch[2].to(device)).to(torch.float)).data.item()
    return acc


####Now let us test the accuracyof our already trained teacher model

In [53]:
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_ckpt).to(device)

In [54]:
accuracy = 0.0
time_taken = 0.0
count = 0
for batch in tqdm(test_loader):
  start_time = perf_counter()
  score = accuracy_score(batch, teacher_model)
  end_time = perf_counter()
  accuracy += score
  time_taken += end_time - start_time

print('\n\n')
print(f"number of samples in each batch is {len(batch[0])}")
print(f'number of batch is {len(test_loader)}')
print(f"accuracy is {accuracy / len(test_loader):.2f}")
print(f'time taken per batch is {time_taken / len(test_loader):.6f}')



  6%|▌         | 3/54 [00:12<03:40,  4.33s/it]




number of samples in each batch is 32
number of batch is 54
accuracy is 0.05
time taken per batch is 0.240102


####On a T5 GPU provided by colab we are able to do inference on each batch in .27 seconds. Let's see if we can match perfocnacena dn reduce inference time for the same test becnh


###We download our student model

In [55]:
student_model = AutoModelForSequenceClassification.from_pretrained(student_model_ckpt, num_labels = 20).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
epochs = 3
learning_rate = 2e-5
entropy_loss = nn.CrossEntropyLoss()
temperature = 2.0
alpha = 0.5
criterion = nn.KLDivLoss(reduction = 'batchmean')
optimizer = optim.Adam(student_model.parameters(), lr = learning_rate)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
for epoch in tqdm(range(epochs), total = epochs):
  #start training loader
  student_model.train()
  training_loss = 0.0
  training_kd_loss = 0.0
  training_accuracy = 0.0
  valid_loss = 0.0
  valid_accuracy = 0.0
  for batch in train_loader:
    optimizer.zero_grad()
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    target_tensors = batch[2].to(device)
    output_stu = student_model(input_ids = input_ids, attention_mask = attention_mask).logits
    loss_ce = entropy_loss(output_stu, target_tensors).data.item()
    #now we extract logits from teacher too!
    with torch.no_grad():
      outputs_teach = teacher_model(input_ids = input_ids, attention_mask = attention_mask)
      outputs_teach_logits = outputs_teach.logits
    loss_kd = temperature ** 2 * criterion(
        F.log_softmax(output_stu /  temperature, dim = -1),
        F.softmax(outputs_teach_logits / temperature, dim = -1)
        )
    loss = alpha * loss_ce + (1. - alpha) * loss_kd
    loss.backward()
    optimizer.step()
    training_kd_loss += loss_kd.data.item()
    training_loss += loss
    training_accuracy += accuracy_score(batch, student_model)

  student_model.eval()
  for batch in valid_loader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    target_tensors = batch[2].to(device)
    output = student_model(input_ids = input_ids, attention_mask = attention_mask)
    loss = criterion(output.logits, target_tensors)
    valid_loss += loss.data.item()
    valid_accuracy += accuracy_score(batch, student_model)

  training_accuracy /= len(train_loader)
  valid_accuracy /= len(valid_loader)
  training_loss /= len(train_loader)
  training_kd_loss /= len(train_loader)
  valid_loss /= len(valid_loader)
  #we would also test accuracy on validation
  print(f"""
    After epochs {epoch + 1},
    training loss (entropy) was {training_loss},
    Kullback-Leibler (KL) divergence loss was {training_kd_loss}
    validation_loss was {valid_loss}.
    training_accuracy {training_accuracy}
    valid_accuracy {valid_accuracy}
  """)




 33%|███▎      | 1/3 [1:42:03<3:24:07, 6123.65s/it]


    After epochs 1, 
    training loss (entropy) was 1.3893439892839503, 
    Kullback-Leibler (KL) divergence loss was 5.052333517710368
    validation_loss was 1.386915099143982. 
    training_accuracy 0.2638333333333333 
    valid_accuracy 0.99675
  


 33%|███▎      | 1/3 [1:49:38<3:39:17, 6578.52s/it]


KeyboardInterrupt: ignored

In [ ]:
student_model.push_to_hub('odunola/student_distillation_model')

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/odunola/bert-yelp-review_full-test-set-only/commit/0c636376ca9508e2f235c9c3fbb144dd37a92fa8', commit_message='Upload BertForSequenceClassification', commit_description='', oid='0c636376ca9508e2f235c9c3fbb144dd37a92fa8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
def get_parameter_count(model):
  num_params = sum(p.numel() for p in model.parameters())
  return num_params

print(f'teacher model has {(get_parameter_count(teacher_model)/1000000):.2f} parameters')
print(f'student model has {(get_parameter_count(student_model)/1000000):.2f} parameters')